In [2]:
import numpy as np
import itertools

Đối với bài này, em không dùng những thư viện sẵn có như `linprog` hay `pulp` mà dùng những kiến thức cơ sở môn Quy hoạch tuyến tính để cài đặt các thuật toán. Chi tiết những kiến thức cơ sở đó là gì và những thuật toán được cài đặt như nào xin Quý thầy cô theo dõi `doc` đi kèm ạ.

In [3]:
def find_extreme_points(a, b, d):
    """Tìm tất cả các điểm cực biên của miền ràng buộc."""
    M = len(a)
    extreme_points = []
    
    # Thêm gốc tọa độ nếu nó thỏa mãn các ràng buộc
    if all(d[i] >= 0 for i in range(M)):
        extreme_points.append((0, 0))
    
    # Giao điểm với các trục tọa độ
    for i in range(M):
        if a[i] != 0:
            x1 = d[i] / a[i]
            if x1 >= 0 and all(a[j] * x1 <= d[j] for j in range(M)):
                extreme_points.append((x1, 0))
        
        if b[i] != 0:
            x2 = d[i] / b[i]
            if x2 >= 0 and all(b[j] * x2 <= d[j] for j in range(M)):
                extreme_points.append((0, x2))
    
    # Giao điểm của các ràng buộc
    for i, j in itertools.combinations(range(M), 2):
        det = a[i] * b[j] - a[j] * b[i]
        
        if det != 0:  # Đường thẳng không song song
            x1 = (d[i] * b[j] - d[j] * b[i]) / det
            x2 = (a[i] * d[j] - a[j] * d[i]) / det  
            
            if x1 >= 0 and x2 >= 0:  # Ràng buộc không âm
                # Kiểm tra xem điểm này có thỏa mãn tất cả các ràng buộc
                if all(a[k] * x1 + b[k] * x2 <= d[k] for k in range(M)):
                    extreme_points.append((x1, x2))
    
    # Loại bỏ các điểm trùng lặp và làm tròn
    unique_points = []
    for p in extreme_points:
        p_rounded = (round(p[0], 10), round(p[1], 10))
        if p_rounded not in [(round(up[0], 10), round(up[1], 10)) for up in unique_points]:
            unique_points.append(p)
    
    return unique_points

Sau đây là $2$ hướng em tìm hiểu để giải quyết vấn đề xác định miền khả thi của bài toán có bị chặn hay không. Đầu tiên là cách ngây thơ (`Naive`) ban đầu. Vì muốn tối thiểu số hướng cần kiểm tra mà đã mắc phải những sai sót với một số bài toán. Ở phần dưới sẽ có ví dụ cho việc này.

In [13]:
def check_bounded_region_naive(a, b, d):
    """Kiểm tra xem miền ràng buộc có bị chặn hay không.
    Tuy nhiên đây là phương án sai do tham lam muốn tối thiểu hóa số hướng kiểm tra."""
    M = len(a)
    
    # Các hướng cơ bản để kiểm tra
    directions = [(1, 0), (0, 1), (1, 1), (-1, 1)]  #  (-1, 1)  có thể không cần thiết
    
    for d1, d2 in directions:
        # Kiểm tra xem hướng (d1, d2) có cho phép mở rộng vô hạn không
        if all(a[i] * d1 + b[i] * d2 <= 0 for i in range(M)):
            return False  # Tìm thấy hướng mở rộng vô hạn
    
    return True  # Không tìm thấy hướng mở rộng vô hạn

Còn đây là phương án sẽ được sử dụng, với độ phức tạp $O(M \log M)$, với $M$ là số ràng buộc của bài toán. Độ phức tạp này không kém nhiều khi so với thuật toán `QuickHull` của hàm `ConvexHull` thuộc thư viện `scipy `, với độ phức tạp $O(N \log N)$, trong đó $N$ là số điểm cực biên thỏa bài toán.

In [17]:
def check_bounded_region(a, b, d):
    """
    Kiểm tra xem miền ràng buộc có bị chặn hay không dựa trên việc phân tích các vector pháp tuyến của các ràng buộc.
    
    Input:
    - a, b: hệ số của các ràng buộc a[i]*x + b[i]*y <= d[i]
    
    Output:
    - True nếu miền bị chặn, False nếu không bị chặn
    """
    import math
    
    M = len(a)
    
    # Tạo danh sách các vector pháp tuyến (bao gồm cả ràng buộc x >= 0, y >= 0)
    normals = [(a[i], b[i]) for i in range(M)]
    
    # Thêm các ràng buộc x >= 0, y >= 0 nếu cần
    contains_x_constraint = False
    contains_y_constraint = False
    
    for nx, ny in normals:
        if nx < 0 and ny == 0:  # Pháp tuyến của x >= 0 là (-1, 0)
            contains_x_constraint = True
        if nx == 0 and ny < 0:  # Pháp tuyến của y >= 0 là (0, -1)
            contains_y_constraint = True
    
    if not contains_x_constraint:
        normals.append((-1, 0))
    if not contains_y_constraint:
        normals.append((0, -1))
    
    # Tính góc của các vector pháp tuyến
    angles = []
    for nx, ny in normals:
        angle = math.atan2(ny, nx)
        angles.append(angle)
    
    # Sắp xếp các góc
    angles.sort()
    
    # Kiểm tra khoảng cách giữa các góc liên tiếp
    for i in range(len(angles)):
        angle1 = angles[i]
        angle2 = angles[(i + 1) % len(angles)]
        
        # Đảm bảo góc nằm trong khoảng [0, 2*pi]
        if angle2 < angle1:
            angle2 += 2 * math.pi
            
        diff = angle2 - angle1
        
        # Nếu có khoảng trống lớn hơn pi, miền không bị chặn
        if diff > math.pi:
            return False
    
    return True

In [5]:
def linear_programming_solver(M, a, b, d, c1, c2, objective_type='minimize'):
    """
    Giải bài toán quy hoạch tuyến tính với các ràng buộc a[i]*x1 + b[i]*x2 <= d[i]
    và hàm mục tiêu F = c1*x1 + c2*x2 (minimize hoặc maximize)
    
    Tham số:
    ---------
    M : int
        Số ràng buộc (M <= 10)
    a : list
        Hệ số của x1 trong các ràng buộc
    b : list
        Hệ số của x2 trong các ràng buộc
    d : list
        Vế phải của các ràng buộc
    c1 : float
        Hệ số của x1 trong hàm mục tiêu
    c2 : float
        Hệ số của x2 trong hàm mục tiêu
    objective_type : str
        'minimize' hoặc 'maximize'
        
    Trả về:
    -------
    Dictionary chứa:
    - extreme_points: Danh sách các điểm cực biên
    - feasible_region_bounded: Boolean cho biết miền ràng buộc có bị chặn hay không
    - optimal_solutions: Dictionary chứa GTNN và GTLN (nếu tồn tại)
    """
    # Kiểm tra đầu vào
    if M > 10 or M <= 0:
        raise ValueError("M phải nằm trong khoảng từ 1 đến 10")
    
    if len(a) != M or len(b) != M or len(d) != M:
        raise ValueError("Độ dài của a, b, và d phải bằng M")
    
    for val in a + b + d:
        if val < -50 or val > 50:
            raise ValueError("Tất cả các hệ số phải nằm trong khoảng [-50, 50]")
    
    # Chuyển đổi sang mảng numpy
    a = np.array(a)
    b = np.array(b)
    d = np.array(d)
    
    # Tìm các điểm cực biên
    extreme_points = find_extreme_points(a, b, d)
    
    # Kiểm tra xem miền ràng buộc có bị chặn hay không
    is_bounded = check_bounded_region(a, b, d)
    
    optimal_solutions = {"GTNN": None, "GTLN": None}  # Mặc định là None
    
    if extreme_points:  # Nếu có miền khả thi
        # Tính giá trị hàm mục tiêu tại từng điểm cực biên
        objective_values = [c1 * x1 + c2 * x2 for (x1, x2) in extreme_points]
        
        if is_bounded:
            min_idx = np.argmin(objective_values)
            max_idx = np.argmax(objective_values)
            optimal_solutions["GTNN"] = (extreme_points[min_idx][0], extreme_points[min_idx][1], objective_values[min_idx])
            optimal_solutions["GTLN"] = (extreme_points[max_idx][0], extreme_points[max_idx][1], objective_values[max_idx])
        else:
            directions = [(1, 0), (0, 1), (1, 1), (-1, 1)]
            unbounded_min = False
            unbounded_max = False
            
            for d1, d2 in directions:
                if all(a[i] * d1 + b[i] * d2 <= 0 for i in range(M)):
                    dot_product = c1 * d1 + c2 * d2
                    if dot_product > 0:
                        unbounded_max = True
                    elif dot_product < 0:
                        unbounded_min = True
            
            min_idx = np.argmin(objective_values)
            max_idx = np.argmax(objective_values)
            
            if not unbounded_min:
                optimal_solutions["GTNN"] = (extreme_points[min_idx][0], extreme_points[min_idx][1], objective_values[min_idx])
            if not unbounded_max:
                optimal_solutions["GTLN"] = (extreme_points[max_idx][0], extreme_points[max_idx][1], objective_values[max_idx])
    
    return {
        "extreme_points": extreme_points if extreme_points else [],
        "feasible_region_bounded": is_bounded,
        "optimal_solutions": optimal_solutions
    }

In [6]:
def input_from_keyboard():
    """Nhập dữ liệu từ bàn phím."""
    try:
        M = int(input("Nhập số điều kiện M (M <= 10): "))
        if M > 10 or M <= 0:
            print("M phải nằm trong khoảng từ 1 đến 10")
            return None
        
        a = []
        b = []
        d = []
        
        print("Nhập các hệ số của điều kiện a[i]*x1 + b[i]*x2 <= d[i]:")
        for i in range(M):
            a_i = float(input(f"Nhập a[{i}]: "))
            b_i = float(input(f"Nhập b[{i}]: "))
            d_i = float(input(f"Nhập d[{i}]: "))
            
            a.append(a_i)
            b.append(b_i)
            d.append(d_i)
        
        c1 = float(input("Nhập hệ số c1 của hàm mục tiêu F = c1*x1 + c2*x2: "))
        c2 = float(input("Nhập hệ số c2 của hàm mục tiêu F = c1*x1 + c2*x2: "))
        
        objective_type = input("Nhập loại mục tiêu (minimize/maximize): ").lower()
        if objective_type not in ['minimize', 'maximize']:
            print("Loại mục tiêu phải là 'minimize' hoặc 'maximize'")
            return None
        
        return M, a, b, d, c1, c2, objective_type
    
    except ValueError:
        print("Lỗi: Vui lòng nhập các giá trị số")
        return None

In [7]:
def output_results(results):
    print("\nKết quả:")
    print("1) Danh sách các điểm cực biên:")
    if results["extreme_points"]:
        for i, point in enumerate(results["extreme_points"]):
            print(f"   Điểm {i+1}: ({point[0]}, {point[1]})")
    else:
        print("   Không có điểm cực biên")
    
    print("\n2) Miền ràng buộc có bị chặn hay không?")
    print(f"   {'Miền ràng buộc bị chặn' if results['feasible_region_bounded'] else 'Miền ràng buộc không bị chặn'}")
    
    print("\n3) GTNN và GTLN tìm được:")
    if results["optimal_solutions"]["GTNN"]:
        x1_min, x2_min, obj_value_min = results["optimal_solutions"]["GTNN"]
        print(f"   GTNN: x1 = {x1_min}, x2 = {x2_min}, F = {obj_value_min}")
    else:
        print("   GTNN: Không tồn tại")
    if results["optimal_solutions"]["GTLN"]:
        x1_max, x2_max, obj_value_max = results["optimal_solutions"]["GTLN"]
        print(f"   GTLN: x1 = {x1_max}, x2 = {x2_max}, F = {obj_value_max}")
    else:
        print("   GTLN: Không tồn tại")

In [ ]:
# def main():
#     print("Chương trình giải bài toán quy hoạch tuyến tính")
#     print("Dạng bài toán: F = c1*x1 + c2*x2 với ràng buộc a[i]*x1 + b[i]*x2 <= d[i]")
    
#     inputs = input_from_keyboard()
#     if inputs:
#         M, a, b, d, c1, c2, objective_type = inputs
#         results = linear_programming_solver(M, a, b, d, c1, c2, objective_type)
#         output_results(results)

# if __name__ == "__main__":
#     main()


# Nếu muốn tự nhập bài toán từ bàn phím thì dùng cái này

Chương trình giải bài toán quy hoạch tuyến tính
Dạng bài toán: F = c1*x1 + c2*x2 với ràng buộc a[i]*x1 + b[i]*x2 <= d[i]
Lỗi: Vui lòng nhập các giá trị số


Trước khi bước vào phần `Test case`, hãy theo dõi một phản ví dụ cho phương pháp ở `check_bounded_region_naive`

In [15]:
def linear_programming_solver_naive(M, a, b, d, c1, c2, objective_type='minimize'):
    """
    Giải bài toán quy hoạch tuyến tính với các ràng buộc a[i]*x1 + b[i]*x2 <= d[i]
    và hàm mục tiêu F = c1*x1 + c2*x2 (minimize hoặc maximize)
    
    Tham số:
    ---------
    M : int
        Số ràng buộc (M <= 10)
    a : list
        Hệ số của x1 trong các ràng buộc
    b : list
        Hệ số của x2 trong các ràng buộc
    d : list
        Vế phải của các ràng buộc
    c1 : float
        Hệ số của x1 trong hàm mục tiêu
    c2 : float
        Hệ số của x2 trong hàm mục tiêu
    objective_type : str
        'minimize' hoặc 'maximize'
        
    Trả về:
    -------
    Dictionary chứa:
    - extreme_points: Danh sách các điểm cực biên
    - feasible_region_bounded: Boolean cho biết miền ràng buộc có bị chặn hay không
    - optimal_solutions: Dictionary chứa GTNN và GTLN (nếu tồn tại)
    """
    # Kiểm tra đầu vào
    if M > 10 or M <= 0:
        raise ValueError("M phải nằm trong khoảng từ 1 đến 10")
    
    if len(a) != M or len(b) != M or len(d) != M:
        raise ValueError("Độ dài của a, b, và d phải bằng M")
    
    for val in a + b + d:
        if val < -50 or val > 50:
            raise ValueError("Tất cả các hệ số phải nằm trong khoảng [-50, 50]")
    
    # Chuyển đổi sang mảng numpy
    a = np.array(a)
    b = np.array(b)
    d = np.array(d)
    
    # Tìm các điểm cực biên
    extreme_points = find_extreme_points(a, b, d)
    
    # Kiểm tra xem miền ràng buộc có bị chặn hay không
    is_bounded = check_bounded_region_naive(a, b, d)
    
    optimal_solutions = {"GTNN": None, "GTLN": None}  # Mặc định là None
    
    if extreme_points:  # Nếu có miền khả thi
        # Tính giá trị hàm mục tiêu tại từng điểm cực biên
        objective_values = [c1 * x1 + c2 * x2 for (x1, x2) in extreme_points]
        
        if is_bounded:
            min_idx = np.argmin(objective_values)
            max_idx = np.argmax(objective_values)
            optimal_solutions["GTNN"] = (extreme_points[min_idx][0], extreme_points[min_idx][1], objective_values[min_idx])
            optimal_solutions["GTLN"] = (extreme_points[max_idx][0], extreme_points[max_idx][1], objective_values[max_idx])
        else:
            directions = [(1, 0), (0, 1), (1, 1), (-1, 1)]
            unbounded_min = False
            unbounded_max = False
            
            for d1, d2 in directions:
                if all(a[i] * d1 + b[i] * d2 <= 0 for i in range(M)):
                    dot_product = c1 * d1 + c2 * d2
                    if dot_product > 0:
                        unbounded_max = True
                    elif dot_product < 0:
                        unbounded_min = True
            
            min_idx = np.argmin(objective_values)
            max_idx = np.argmax(objective_values)
            
            if not unbounded_min:
                optimal_solutions["GTNN"] = (extreme_points[min_idx][0], extreme_points[min_idx][1], objective_values[min_idx])
            if not unbounded_max:
                optimal_solutions["GTLN"] = (extreme_points[max_idx][0], extreme_points[max_idx][1], objective_values[max_idx])
    
    return {
        "extreme_points": extreme_points if extreme_points else [],
        "feasible_region_bounded": is_bounded,
        "optimal_solutions": optimal_solutions
    }

def main():
    test_case = (2, [-2, 2], [3, -4], [10, 5], 1, 1, "minimize")
    M, a, b, d, c1, c2, objective_type = test_case
    results = linear_programming_solver_naive(M, a, b, d, c1, c2, objective_type)
    print("Kết quả:", results)

if __name__ == "__main__":
    main()

Kết quả: {'extreme_points': [(0, 0), (0, 3.3333333333333335), (2.5, 0)], 'feasible_region_bounded': True, 'optimal_solutions': {'GTNN': (0, 0, 0), 'GTLN': (0, 3.3333333333333335, 3.3333333333333335)}}


Ở đây mặc dù kết quả đưa ra là `'feasible_region_bounded': True`, trên thực tế miền khả thi này không bị chặn. Do hướng cực biên thỏa của hệ các ràng buộc này là $(2,1)$ chứ không thuộc một trong số các hướng mà hàm kiểm tra. 

In [9]:
import matplotlib.pyplot as plt
import os

Sau đây sẽ là $25$ test case để kiểm tra những thuật toán vừa cài, $10$ test case đầu là những bài toán đơn giản với $2$ đến $3$ ràng buộc. Hai test case thứ $11$ và $12$ là mẫu từ bài toán. Từ test case thứ $13$ trở đi sẽ là những bài toán phức tạp hơn, từ $4$ đến $10$ ràng buộc.

Kết quả những test case này sẽ được lưu vào folder `result`, mỗi test case sẽ có một file txt hiển thị kết quả cho $3$ câu hỏi của bài toán, và một folder chứa ảnh visualization dưới dạng đồ thị.

In [10]:
# Danh sách 10 test case, ở file txt kết quả sẽ có max và min
test_cases = [
    (2, [1, 2], [2, 1], [4, 3], 3, 5, "maximize"),
    (3, [1, 3, 2], [2, 1, 3], [5, 4, 6], 2, 4, "minimize"),
    (2, [2, -1], [1, 1], [6, 2], 1, 2, "maximize"),
    (2, [-1, -1], [1, -1], [2, 3], 1, 1, "maximize"),
    (2, [3, 2], [1, 4], [12, 10], 5, 2, "maximize"),
    (3, [-1, 1, -2], [-1, -1, 1], [5, 4, 3], 2, 3, "minimize"),
    (2, [-2, 1], [1, -1], [4, 2], 3, 4, "maximize"),
    (2, [-1, -2], [-2, -1], [5, 6], 1, 2, "maximize"),
    (2, [1, 1], [2, -1], [5, 3], 4, 3, "maximize"),
    (3, [1, -1, -2], [2, -1, 1], [4, 3, 5], 1, 5, "minimize"),
    (3, [1, 0, 1], [1, 1, 2], [5, 2, 6], 30, 50, "maximize"),
    (2, [-2, -2], [-1, -5], [-14, -30], 4, 3, "minimize"),
    (4, [-3, -1, -1, 0], [2, -1, 0, -1], [-6, -12, -5, -4], -6, -5, "minimize"),
    (5, [1, 2, 3, 4, 5], [2, 1, 3, 2, 1], [4, 3, 5, 6, 7], 3, 5, "maximize"),
    (6, [1, 3, 2, 4, 5, 6], [2, 1, 3, 2, 1, 2], [5, 4, 6, 7, 8, 9], 2, 4, "minimize"),
    (4, [2, -1, 3, 4], [1, 1, 2, 3], [6, 2, 7, 8], 1, 2, "maximize"),
    (3, [-1, -1, 2], [1, -1, 3], [2, 3, 4], 1, 1, "maximize"),
    (5, [3, 2, 1, 4, 5], [1, 4, 3, 2, 1], [12, 10, 8, 7, 6], 5, 2, "maximize"),
    (7, [-1, 1, -2, 3, 4, 5, 6], [-1, -1, 1, 2, 3, 4, 5], [5, 4, 3, 6, 7, 8, 9], 2, 3, "minimize"),
    (4, [-2, 1, 3, 4], [1, -1, 2, 3], [4, 2, 5, 6], 3, 4, "maximize"),
    (6, [-1, -2, 3, 4, 5, 6], [-2, -1, 2, 3, 4, 5], [5, 6, 7, 8, 9, 10], 1, 2, "maximize"),
    (5, [1, 1, 2, 3, 4], [2, -1, 3, 4, 5], [5, 3, 6, 7, 8], 4, 3, "maximize"),
    (8, [1, -1, -2, 3, 4, 5, 6, 7], [2, -1, 1, 2, 3, 4, 5, 6], [4, 3, 5, 6, 7, 8, 9, 10], 1, 5, "minimize"),
    (9, [1, 0, 1, 2, 3, 4, 5, 6, 7], [1, 1, 2, 3, 4, 5, 6, 7, 8], [5, 2, 6, 7, 8, 9, 10, 11, 12], 30, 50, "maximize"),
    (10, [-2, -2, 1, 2, 3, 4, 5, 6, 7, 8], [-1, -5, 2, 3, 4, 5, 6, 7, 8, 9], [-14, -30, 5, 6, 7, 8, 9, 10, 11, 12], 4, 3, "minimize")
]


In [11]:
def visualize(a, b, d, c1=None, c2=None, optimal_solution=None, save_path=None):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    if not a or not b or not d:
        ax.set_xlabel('x1', fontsize=12)
        ax.set_ylabel('x2', fontsize=12)
        ax.set_title('Miền khả thi', fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.7)
    else:
        extreme_points = find_extreme_points(a, b, d)
        if extreme_points:
            points = np.array(extreme_points)
            max_x = max(5, np.max(points[:, 0]) * 1.2)
            max_y = max(5, np.max(points[:, 1]) * 1.2)
            
            x = np.linspace(0, max_x, 1000)
            for i in range(len(a)):
                if b[i] != 0:
                    y = (d[i] - a[i] * x) / b[i]
                    mask = (y >= 0) & (y <= max_y * 1.2)
                    ax.plot(x[mask], y[mask], label=f"Constraint {i+1}")
                elif a[i] != 0:
                    x_val = d[i] / a[i]
                    if 0 <= x_val <= max_x:
                        ax.axvline(x=x_val, label=f"Constraint {i+1}")
            
            if len(extreme_points) >= 3:
                center = np.mean(points, axis=0)
                angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
                sorted_indices = np.argsort(angles)
                sorted_points = points[sorted_indices]
                sorted_points = np.vstack([sorted_points, sorted_points[0]])
                ax.fill(sorted_points[:, 0], sorted_points[:, 1], 
                        alpha=0.3, color='skyblue', edgecolor='blue', label='Feasible Region')
            
            ax.scatter(points[:, 0], points[:, 1], color='red', s=50, zorder=5)
            for i, point in enumerate(extreme_points):
                ax.annotate(f"P{i+1}", xy=(point[0], point[1]), 
                           xytext=(point[0]+0.1, point[1]+0.1))
            
            # Kiểm tra optimal_solution trước khi truy cập
            if c1 is not None and c2 is not None and optimal_solution is not None:
                x1_opt, x2_opt, f_opt = optimal_solution
                if c2 != 0:
                    x = np.linspace(0, max_x, 100)
                    y = (f_opt - c1 * x) / c2
                    mask = (y >= 0) & (y <= max_y)
                    ax.plot(x[mask], y[mask], 'g--', linewidth=2, label=f"F = {f_opt:.2f}")
                ax.scatter([x1_opt], [x2_opt], color='green', s=100, zorder=10, 
                          marker='*', label="Điểm tối ưu")
                ax.annotate(f"Tối ưu ({x1_opt:.2f}, {x2_opt:.2f})",
                           xy=(x1_opt, x2_opt), xytext=(x1_opt+0.5, x2_opt+0.5),
                           arrowprops=dict(facecolor='black', shrink=0.05), zorder=11)
            
            ax.set_xlim(0, max_x)
            ax.set_ylim(0, max_y)
            ax.legend(loc='upper right')
    
    ax.set_xlabel('x1', fontsize=12)
    ax.set_ylabel('x2', fontsize=12)
    ax.set_title('Miền khả thi', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

In [19]:
# Tạo thư mục result nếu chưa tồn tại
if not os.path.exists("result"):
    os.makedirs("result")

for i, inputs in enumerate(test_cases):
    print(f"\n🔹 Test case {i+1}:")
    M, a, b, d, c1, c2, objective_type = inputs
    
    # Giải bài toán
    try:
        results = linear_programming_solver(M, a, b, d, c1, c2, objective_type)
    except Exception as e:
        print(f"Lỗi khi giải bài toán: {e}")
        continue
    
    # Tạo thư mục cho test case
    test_case_dir = os.path.join("result", f"test_case_{i+1}")
    if not os.path.exists(test_case_dir):
        os.makedirs(test_case_dir)
    
    # Lấy GTNN và GTLN từ results
    optimal_min_solution = results["optimal_solutions"]["GTNN"]
    optimal_max_solution = results["optimal_solutions"]["GTLN"]
    
    # Xác định nghiệm tối ưu theo objective_type
    optimal_solution = optimal_max_solution if objective_type == "maximize" else optimal_min_solution
    optimal_type = "GTLN" if objective_type == "maximize" else "GTNN"
    
    # Lưu file txt với cả GTNN và GTLN
    with open(os.path.join(test_case_dir, f"test_case_{i+1}.txt"), "w", encoding="utf-8") as f:
        f.write(f"Test Case {i+1}:\n")
        f.write(f"Số ràng buộc M: {M}\n")
        f.write("Ràng buộc:\n")
        for j in range(M):
            f.write(f"  {a[j]}*x1 + {b[j]}*x2 <= {d[j]}\n")
        f.write(f"Hàm mục tiêu: F = {c1}*x1 + {c2}*x2 ({objective_type})\n")
        f.write("\nKết quả:\n")
        f.write("1) Danh sách các điểm cực biên:\n")
        if results["extreme_points"]:
            for k, point in enumerate(results["extreme_points"]):
                f.write(f"   Điểm {k+1}: ({point[0]}, {point[1]})\n")
        else:
            f.write("   Không có điểm cực biên\n")
        f.write("\n2) Miền ràng buộc có bị chặn hay không?\n")
        f.write(f"   {'Miền ràng buộc bị chặn' if results['feasible_region_bounded'] else 'Miền ràng buộc không bị chặn'}\n")
        f.write("\n3) GTNN và GTLN tìm được:\n")
        if optimal_min_solution:
            x1_min, x2_min, obj_value_min = optimal_min_solution
            f.write(f"   GTNN: x1 = {x1_min}, x2 = {x2_min}, F = {obj_value_min}\n")
        else:
            f.write("   GTNN: Không tồn tại\n")
        if optimal_max_solution:
            x1_max, x2_max, obj_value_max = optimal_max_solution
            f.write(f"   GTLN: x1 = {x1_max}, x2 = {x2_max}, F = {obj_value_max}\n")
        else:
            f.write("   GTLN: Không tồn tại\n")
        f.write(f"\nNghiệm tối ưu theo yêu cầu ({objective_type}): ")
        if optimal_solution:
            x1, x2, obj_value = optimal_solution
            f.write(f"x1 = {x1}, x2 = {x2}, F = {obj_value} ({optimal_type})\n")
        else:
            f.write("Không tồn tại\n")
    
    # In kết quả ra màn hình
    output_results(results)


🔹 Test case 1:

Kết quả:
1) Danh sách các điểm cực biên:
   Điểm 1: (0, 0)
   Điểm 2: (0, 2.0)
   Điểm 3: (1.5, 0)
   Điểm 4: (0.6666666666666666, 1.6666666666666667)

2) Miền ràng buộc có bị chặn hay không?
   Miền ràng buộc bị chặn

3) GTNN và GTLN tìm được:
   GTNN: x1 = 0, x2 = 0, F = 0
   GTLN: x1 = 0.6666666666666666, x2 = 1.6666666666666667, F = 10.333333333333334

🔹 Test case 2:

Kết quả:
1) Danh sách các điểm cực biên:
   Điểm 1: (0, 0)
   Điểm 2: (1.3333333333333333, 0)
   Điểm 3: (0, 2.0)
   Điểm 4: (0.8571428571428571, 1.4285714285714286)

2) Miền ràng buộc có bị chặn hay không?
   Miền ràng buộc bị chặn

3) GTNN và GTLN tìm được:
   GTNN: x1 = 0, x2 = 0, F = 0
   GTLN: x1 = 0, x2 = 2.0, F = 8.0

🔹 Test case 3:

Kết quả:
1) Danh sách các điểm cực biên:
   Điểm 1: (0, 0)
   Điểm 2: (3.0, 0)
   Điểm 3: (0, 2.0)
   Điểm 4: (1.3333333333333333, 3.3333333333333335)

2) Miền ràng buộc có bị chặn hay không?
   Miền ràng buộc bị chặn

3) GTNN và GTLN tìm được:
   GTNN: x1 = 0, x2 

Kết quả của các `test case` trong các file txt đã được đối chiếu với kết quả từ `Maple`, Thầy có thể kiểm tra ở trong tệp mw đính kèm ạ.